# Семантическое ядро с интеграцией OpenBnB MCP Server

Этот блокнот демонстрирует, как использовать семантическое ядро с реальным сервером OpenBnB MCP для поиска настоящих вариантов размещения Airbnb с помощью MCPStdioPlugin. Для доступа к LLM используется Azure AI Foundry. Чтобы настроить переменные окружения, вы можете следовать [уроку по настройке](/00-course-setup/README.md).


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Создание подключения плагина MCP

Мы подключимся к [серверу OpenBnB MCP](https://github.com/openbnb-org/mcp-server-airbnb) с использованием MCPStdioPlugin. Этот сервер предоставляет функциональность поиска Airbnb через пакет @openbnb/mcp-server-airbnb.


## Создание клиента

В этом примере мы будем использовать Azure AI Foundry для доступа к LLM. Убедитесь, что переменные окружения настроены правильно.


## Настройка окружения

Настройте параметры Azure OpenAI. Убедитесь, что у вас установлены следующие переменные окружения:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Понимание интеграции OpenBnB MCP

Этот ноутбук подключается к **реальному серверу OpenBnB MCP**, который предоставляет функционал поиска Airbnb.

### Как это работает:

1. **MCPStdioPlugin**: Использует стандартное ввод/вывод для общения с сервером MCP
2. **Реальный NPM пакет**: Загружает и запускает `@openbnb/mcp-server-airbnb` через npx
3. **Живые данные**: Возвращает актуальные данные о недвижимости Airbnb из их API
4. **Обнаружение функций**: Агент автоматически обнаруживает доступные функции сервера MCP

### Доступные функции:

Сервер OpenBnB MCP обычно предоставляет:
- **search_listings** - Поиск недвижимости Airbnb по местоположению и критериям
- **get_listing_details** - Получение подробной информации о конкретной недвижимости
- **check_availability** - Проверка доступности на определенные даты
- **get_reviews** - Получение отзывов о недвижимости
- **get_host_info** - Получение информации о хозяевах недвижимости

### Предварительные требования:

- **Node.js**, установленный на вашем компьютере
- **Интернет-соединение** для загрузки пакета сервера MCP
- **NPX** (поставляется с Node.js)

### Тестирование подключения:

Вы можете протестировать сервер MCP вручную, запустив:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Это загрузит и запустит сервер OpenBnB MCP, к которому Semantic Kernel затем подключается для получения реальных данных Airbnb.


## Запуск агента с сервером OpenBnB MCP

Теперь мы запустим AI-агента, который подключается к серверу OpenBnB MCP, чтобы искать реальные варианты жилья на Airbnb в Стокгольме для 2 взрослых и 1 ребенка. Вы можете изменить список `user_inputs`, чтобы настроить критерии поиска.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Резюме
Поздравляем! Вы успешно создали AI-агента, который интегрируется с поиском жилья в реальном мире, используя протокол Model Context Protocol (MCP):

## Используемые технологии:
- Semantic Kernel - Для создания интеллектуальных агентов с Azure OpenAI
- Azure AI Foundry - Для возможностей LLM и завершения чатов
- MCP (Model Context Protocol) - Для стандартизированной интеграции инструментов
- OpenBnB MCP Server - Для реального поиска жилья на Airbnb
- Node.js/NPX - Для запуска внешнего MCP-сервера

## Чему вы научились:
- Интеграция MCP: Подключение агентов Semantic Kernel к внешним MCP-серверам
- Доступ к данным в реальном времени: Поиск реальных объектов Airbnb через живые API
- Протокольная коммуникация: Использование stdio для связи между агентом и MCP-сервером
- Обнаружение функций: Автоматическое обнаружение доступных функций на MCP-серверах
- Потоковые ответы: Захват и логирование вызовов функций в реальном времени
- Рендеринг HTML: Форматирование ответов агента с помощью стилизованных таблиц и интерактивных элементов

## Следующие шаги:
- Интеграция дополнительных MCP-серверов (погода, рейсы, рестораны)
- Создание системы с несколькими агентами, объединяющей MCP и A2A протоколы
- Разработка собственных MCP-серверов для ваших источников данных
- Реализация постоянной памяти разговоров между сессиями
- Размещение агента в Azure Functions с оркестрацией MCP-сервера
- Добавление аутентификации пользователей и возможностей бронирования



---

**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, имейте в виду, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
